<a href="https://colab.research.google.com/github/utsavajay/projects/blob/main/facebook_post_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.3 MB/s eta 0:00:00


In [17]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.2 MB/s eta 0:00:00


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.8 MB/s eta 0:00:00


In [58]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis",  num_labels = 3)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [14]:
import pandas as pd
from datasets import Dataset
from transformers import TrainingArguments

In [63]:
lab = pd.read_csv('lab.txt', sep='\t')
unlab =  pd.read_csv('unlab.txt', sep='\t')

In [64]:
lab['label'] = 0
lab.loc[lab['Complaint']==1, 'label'] = 1
lab.loc[lab['Feedback']==1, 'label'] = 2

In [65]:
lab['label'].value_counts()

1    4255
0    2062
2    1644
Name: label, dtype: int64

In [66]:
lab = lab[['message', 'label']]

In [67]:
train_ds = Dataset.from_pandas(lab, split="train")
test_ds = Dataset.from_pandas(unlab, split="test")

In [68]:
def tokenize_function(examples):
    return tokenizer(examples["message"], padding="max_length", truncation=True)


train_ds = train_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/7961 [00:00<?, ? examples/s]

Map:   0%|          | 0/2039 [00:00<?, ? examples/s]

In [49]:
training_args = TrainingArguments(output_dir="test_trainer")

In [50]:
import evaluate
import numpy as np 

f1_metric = evaluate.load("f1")

In [51]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return f1_metric.compute(predictions=predictions, references=labels)

In [52]:
from transformers import TrainingArguments, Trainer


In [69]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics
)

In [71]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.480000
1000,0.397600
1500,0.272800
2000,0.229100
2500,0.111900


TrainOutput(global_step=2988, training_loss=0.27010543470880594, metrics={'train_runtime': 642.6102, 'train_samples_per_second': 37.166, 'train_steps_per_second': 4.65, 'total_flos': 1570984438899456.0, 'train_loss': 0.27010543470880594, 'epoch': 3.0})

In [73]:
pred = trainer.predict(test_dataset = test_ds)

In [79]:
pred.predictions

array([[ 4.694942 , -1.9667948, -2.941551 ],
       [ 4.6935587, -1.9390322, -2.967416 ],
       [ 4.6863575, -1.9105619, -2.993317 ],
       ...,
       [-3.4446921,  4.384702 , -1.0530233],
       [-2.8058279,  4.905142 , -2.163434 ],
       [-2.630853 ,  4.231403 , -1.7846854]], dtype=float32)

In [81]:
pred1 = [np.argmax(item) for item in pred.predictions]

In [83]:
unlab['pred1']  = pred1

In [86]:
unlab

,postId,message,pred1
0,108381603303_10151119973393304,Love. It. To,0
1,115568331790246_371841206162956,NICE,0
2,115568331790246_515044031842672,Congrats,0
3,147285781446_10151010892176447,Awesome!,0
4,159616034235_10150639103634236,Award,0
...,...,...,...
2034,179590995428478_422375854483323,"you guys are terrible, holding goverment check...",1
2035,125472670805257_525103854175468,as i platinum elite member of delta and a loya...,1
2036,179590995428478_377568608964048,Really?,1
2037,179590995428478_341070505947192,Horrible decision.,1


In [94]:
unlab1 = pd.concat([unlab, pd.get_dummies(unlab['pred1'])] , axis =1)

In [96]:
unlab1 = unlab1[['postId', 0,1,2]]

In [98]:
unlab1.columns = ['postId', 'Appreciation_pred','Complaint_pred','Feedback_pred']

In [101]:
unlab1.shape

(2039, 4)

In [104]:
unlab1.columns = ['postId','Appreciation_pred','Complaint_pred','Feedback_pred']

In [105]:
unlab1.to_csv('test1.csv')

In [123]:
training_args1 = TrainingArguments(output_dir="test_trainer",     optim = 'adamw_torch' , num_train_epochs = 3,learning_rate = 1e-5)
trainer1 = Trainer(
    model=model,
    args=training_args1,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics

)

In [124]:
trainer1.train()

Step,Training Loss
500,0.018000
1000,0.017800
1500,0.013200
2000,0.011500
2500,0.001200


TrainOutput(global_step=2988, training_loss=0.011216606461060255, metrics={'train_runtime': 667.1831, 'train_samples_per_second': 35.797, 'train_steps_per_second': 4.479, 'total_flos': 1570984438899456.0, 'train_loss': 0.011216606461060255, 'epoch': 3.0})

In [125]:
arr = trainer1.predict(test_dataset = test_ds)

In [126]:
def file_gen(arr):
    df= pd.read_csv('unlab.txt', sep = '\t')
    p = [np.argmax(item) for item in arr.predictions]
    df['pred1']  = p
    df1 = pd.concat([df, pd.get_dummies(df['pred1'])] , axis =1)
    df1 = df1[['postId', 0,1,2]]
    df1.columns = ['postId', 'Appreciation_pred','Complaint_pred','Feedback_pred']
    return df1

In [127]:
x = file_gen(arr)

In [128]:
x.to_csv('test4.csv')